# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key
#jupyter nbextension enable --py gmaps

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_file = "../output_data/cities.csv"
city_weather_df = pd.read_csv(city_weather_file)
#print(city_weather_df)
display(city_weather_df)
print(city_weather_df.count())

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Abaza,100,RU,1584468904,87,52.66,90.09,29.39,4.41
1,Jodhpur,17,IN,1584468904,22,26.29,73.03,75.65,10.49
2,Ancud,40,CL,1584468904,67,-41.87,-73.82,60.80,6.93
3,Albany,17,AU,1584468904,81,-35.02,117.89,53.60,10.29
4,Tiksi,84,RU,1584468905,88,71.69,128.87,-30.73,8.37
...,...,...,...,...,...,...,...,...,...
528,Kozhva,82,RU,1584468987,89,65.11,57.05,21.65,10.71
529,Kazanskoye,50,RU,1584468987,82,55.64,69.23,26.33,17.94
530,Caravelas,9,BR,1584468689,73,-17.71,-39.25,81.77,8.84
531,Eureka,90,US,1584468863,70,40.80,-124.16,46.40,6.93


City          533
Cloudiness    533
Country       531
Date          533
Humidity      533
Lat           533
Lng           533
Max Temp      533
Wind Speed    533
dtype: int64


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '1000px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
intensity = city_weather_df['Humidity'].max()
heatmap_layer = gmaps.heatmap_layer(
    city_weather_df[['Lat', 'Lng']], weights=city_weather_df['Humidity'],
    max_intensity=intensity, point_radius=5.0
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
condition1 = (city_weather_df['Max Temp'] < 80) & (city_weather_df['Max Temp'] > 70)
condition2 = city_weather_df['Wind Speed'] < 10
condition3 = city_weather_df['Cloudiness'] == 0
ideal_weather_df = city_weather_df[(condition1) & (condition2) & (condition3)]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index(drop=True)
print(ideal_weather_df)

            City  Cloudiness Country        Date  Humidity    Lat     Lng  \
0           Lebu           0      CL  1584468929        52 -37.62  -73.65   
1          Umluj           0      SA  1584468958        50  25.02   37.27   
2     Bouloupari           0      NC  1584468962        83 -21.87  166.05   
3  Ahmadpur East           0      PK  1584468966        22  29.14   71.26   
4      Inhambane           0      MZ  1584468970        64 -23.86   35.38   
5     Srīvardhan           0      IN  1584468972        56  18.03   73.02   
6       Kopāganj           0      IN  1584468973        40  26.02   83.57   
7         Merowe           0      SD  1584468984        13  18.48   31.82   
8  Marsh Harbour           0      BS  1584468984        67  26.54  -77.06   

   Max Temp  Wind Speed  
0     70.07        9.31  
1     72.41        6.20  
2     71.60        2.24  
3     79.25        7.23  
4     75.20        6.93  
5     78.53        5.77  
6     70.43        5.66  
7     79.97        8.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
 # Create DataFrame 
data = {'City':ideal_weather_df['City'], 'Country':ideal_weather_df['Country'], 'Lat':ideal_weather_df['Lat'], 'Lng':ideal_weather_df['Lng']}
hotel_df = pd.DataFrame(data) 
hotel_df['Hotel Name'] = ""
# set up a parameters dictionary
target_search = "hotel"
target_radius = 5000
#target_type = "restaurant"
params = {
    "location": "",
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
    }
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in ideal_weather_df.iterrows():
    lat = str(row['Lat'])
    lng = str(row['Lng'])
    target_cords = lat+", "+lng
    #print(target_cords)
    params['location'] = target_cords
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()    
    # extract results
    places_results = response['results']
    #print(places_results)
    try:
        hotel_name = places_results[0]['name']
        print(f"First Hotel is {hotel_name}.")
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("------------------------")

print(hotel_df)

First Hotel is Hotel Plaza Lebu.
------------------------
First Hotel is عروس الشاطئ للوحدات السكنية المفروشة.
------------------------
First Hotel is Zianu lodge.
------------------------
First Hotel is Hotel Pearl Resort.
------------------------
First Hotel is Hotel Casa Do Capitão.
------------------------
First Hotel is Tranquil Beach Resort.
------------------------
First Hotel is Tathagat Vatika.
------------------------
First Hotel is Merowe Tourist Village.
------------------------
First Hotel is Abaco Beach Resort.
------------------------
            City Country    Lat     Lng                            Hotel Name
0           Lebu      CL -37.62  -73.65                      Hotel Plaza Lebu
1          Umluj      SA  25.02   37.27  عروس الشاطئ للوحدات السكنية المفروشة
2     Bouloupari      NC -21.87  166.05                           Zianu lodge
3  Ahmadpur East      PK  29.14   71.26                    Hotel Pearl Resort
4      Inhambane      MZ -23.86   35.38               

In [6]:
# Add marker layer ontop of heat map
#markers = gmaps.marker_layer(hotel_df[['Lat', 'Lng']])
# Add the layer to the map
#fig.add_layer(markers)
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…